# Coleta de Dados WeRateDoga

In [ ]:
import pandas as pd
import requests
import tweepy
import json

## Config tweepy

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## Get twitter-archive-enhanced

In [ ]:
df_tweets =  pd.read_csv("../Data/twitter-archive-enhanced.csv")

## Get image_predictions

In [ ]:
url = " https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
r = requests.get(url)
with open("../Data/image_predictions.tsv", "wb") as code:
     code.write(r.content)
df_images_predictions = pd.read_csv("../Data/image_predictions.tsv",sep='\t')

## Using tweepy for get tweet data

In [ ]:
data_twitter = []
for code in df_images_predictions.tweet_id:
    try:
        status_data = api.get_status(code,tweet_mode='extended')
        data_twitter.append(status_data._json)
        print(str(code)+" - encontrado")
    except:
        print(str(code)+" - não encontrado")

## Save downloaded jsons

In [ ]:
with open("../Data/tweet_json.txt", "w") as file:
    json.dump(data_twitter,file)


## Load downloaded jsons

In [ ]:
with open("../Data/tweet_json.txt") as tweet_json:  
    data_tweets_json = json.load(tweet_json)

## Creating dictionary downloaded data with tweepy

In [ ]:
tweets_data = []
key = 0
for data in data_tweets_json:
    tweets_data.append({}) 
    tweets_data[key]['id'] = data['id']
    tweets_data[key]['favorite_count'] = data['favorite_count']
    tweets_data[key]['retweet_count'] = data['retweet_count']
    key = key+1

## Converting dictionary to dataframe

In [ ]:
df_tweets_data = pd.DataFrame(tweets_data)

In [ ]:
df_tweets_data.head(5)